In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# load data
url = 'https://raw.githubusercontent.com/shloke01/student-performance-prediction/main/'

dataset_df = pd.read_csv(url + "StudentPerformancePred.csv")
labels = pd.read_csv(url + "train_labels.csv")

dataset_df = dataset_df.rename(columns={'session_id': 'session'})

labels['session'] = labels.session_id.apply(lambda x: int(x.split('_')[0]))
labels['q'] = labels.session_id.apply(lambda x: int(x.split('_')[-1][1:]))


#print(dataset_df.head())
#print(labels.head())

In [2]:
# Split the dataset based on the level_group variable
level_group_A = dataset_df.loc[dataset_df['level_group'] == '0-4']
level_group_B = dataset_df.loc[dataset_df['level_group'] == '5-12']
level_group_C = dataset_df.loc[dataset_df['level_group'] == '13-22']

# Split the labels DataFrame based on the question groupings
labels_A = labels[labels['q'] <= 3]
labels_B = labels[(labels['q'] > 3) & (labels['q'] <= 13)]
labels_C = labels[labels['q'] > 13]

# Merge the datasets and labels based on the corresponding level groups
merged_dataset_A = level_group_A.merge(labels_A[['session', 'correct', 'q']], left_on='session', right_on='session')
merged_dataset_B = level_group_B.merge(labels_B[['session', 'correct', 'q']], left_on='session', right_on='session')
merged_dataset_C = level_group_C.merge(labels_C[['session', 'correct', 'q']], left_on='session', right_on='session')

print(merged_dataset_C.head(15))
#print(labels.head())

              session level_group  event_name_nunique  fqid_nunique  \
0   20090312431273200       13-22                  10            49   
1   20090312431273200       13-22                  10            49   
2   20090312431273200       13-22                  10            49   
3   20090312431273200       13-22                  10            49   
4   20090312431273200       13-22                  10            49   
5   20090312433251036       13-22                  11            73   
6   20090312433251036       13-22                  11            73   
7   20090312433251036       13-22                  11            73   
8   20090312433251036       13-22                  11            73   
9   20090312433251036       13-22                  11            73   
10  20090312455206810       13-22                  11            47   
11  20090312455206810       13-22                  11            47   
12  20090312455206810       13-22                  11            47   
13  20

In [3]:
# Assuming train_A, train_B, and train_C are the three datasets you want to merge
merged_df = pd.concat([merged_dataset_A, merged_dataset_B, merged_dataset_C])

merged_df = merged_df.sort_values(by=['session', 'q'])


print(merged_df.head(20))
#print(labels.head())

             session level_group  event_name_nunique  fqid_nunique  \
0  20090312431273200         0-4                  10            30   
1  20090312431273200         0-4                  10            30   
2  20090312431273200         0-4                  10            30   
0  20090312431273200        5-12                  10            39   
1  20090312431273200        5-12                  10            39   
2  20090312431273200        5-12                  10            39   
3  20090312431273200        5-12                  10            39   
4  20090312431273200        5-12                  10            39   
5  20090312431273200        5-12                  10            39   
6  20090312431273200        5-12                  10            39   
7  20090312431273200        5-12                  10            39   
8  20090312431273200        5-12                  10            39   
9  20090312431273200        5-12                  10            39   
0  20090312431273200

In [4]:
from sklearn.model_selection import train_test_split

# Separate the target variable and features
y = merged_df['correct']
X = merged_df.drop('correct', axis=1)

# Split the data into training and testing sets
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2)